# 4. Indexes

## Exercise 21 - Parking tickets

In [207]:
# load in CSV, specify columns
tickets = pd.read_csv(r'D:\Documents\Data Analysis\Datasets\pandas-workout-data\data\nyc-parking-violations-2020.csv',
                     usecols=['Date First Observed', 'Plate ID', 'Registration State', 'Issue Date', 'Vehicle Make', 'Street Name', 'Vehicle Color'])
tickets.head()

,Plate ID,Registration State,Issue Date,Vehicle Make,Street Name,Date First Observed,Vehicle Color
0,J58JKX,NJ,05/08/1972 12:00:00 AM,HONDA,43 ST,0,BK
1,KRE6058,PA,08/29/1977 12:00:00 AM,ME/BE,UNION ST,0,BLK
2,444326R,NJ,10/03/1988 12:00:00 AM,LEXUS,CLERMONT AVENUE,0,BLACK
3,F728330,OH,01/03/1990 12:00:00 AM,CHEVR,DIVISION AVE,0,NaN
4,FMY9090,NY,02/14/1990 12:00:00 AM,JEEP,GRAND ST,0,GREY


In [208]:
# manually convert to datetime, specify format
tickets['Issue Date'] = pd.to_datetime(tickets['Issue Date'], format='%m/%d/%Y %I:%M:%S %p')
tickets.head()

,Plate ID,Registration State,Issue Date,Vehicle Make,Street Name,Date First Observed,Vehicle Color
0,J58JKX,NJ,1972-05-08,HONDA,43 ST,0,BK
1,KRE6058,PA,1977-08-29,ME/BE,UNION ST,0,BLK
2,444326R,NJ,1988-10-03,LEXUS,CLERMONT AVENUE,0,BLACK
3,F728330,OH,1990-01-03,CHEVR,DIVISION AVE,0,NaN
4,FMY9090,NY,1990-02-14,JEEP,GRAND ST,0,GREY


In [209]:
# set 'Issue Date' as index
tickets = tickets.set_index('Issue Date')
tickets.head()

,Plate ID,Registration State,Vehicle Make,Street Name,Date First Observed,Vehicle Color
Issue Date,,,,,,
1972-05-08,J58JKX,NJ,HONDA,43 ST,0,BK
1977-08-29,KRE6058,PA,ME/BE,UNION ST,0,BLK
1988-10-03,444326R,NJ,LEXUS,CLERMONT AVENUE,0,BLACK
1990-01-03,F728330,OH,CHEVR,DIVISION AVE,0,NaN
1990-02-14,FMY9090,NY,JEEP,GRAND ST,0,GREY


In [210]:
# January 2, 2020, 3 most ticketed makes
tickets.loc['2020-01-02', 'Vehicle Make'].value_counts().head(3)

Vehicle Make
TOYOT    3829
HONDA    3593
FORD     3164
Name: count, dtype: int64

In [211]:
# June 1, 2020, 5 streets w/ most tickets
tickets.loc['2020-06-01', 'Street Name'].value_counts().head()

Street Name
WB CONDUIT BLVD @ LI    1068
SB WEST ST @ LEROY S     335
EB HORACE HARDING EX     273
EB QUEENS BLVD @ 82N     245
WB ATLANTIC AVE @ CL     229
Name: count, dtype: int64

In [212]:
# reset index (while keeping it)
tickets = tickets.reset_index()

# set color as index
tickets = tickets.set_index('Vehicle Color')
tickets.head()

,Issue Date,Plate ID,Registration State,Vehicle Make,Street Name,Date First Observed
Vehicle Color,,,,,,
BK,1972-05-08,J58JKX,NJ,HONDA,43 ST,0
BLK,1977-08-29,KRE6058,PA,ME/BE,UNION ST,0
BLACK,1988-10-03,444326R,NJ,LEXUS,CLERMONT AVENUE,0
NaN,1990-01-03,F728330,OH,CHEVR,DIVISION AVE,0
GREY,1990-02-14,FMY9090,NY,JEEP,GRAND ST,0


In [213]:
# most common makes either red/blue
colors = pd.Series(tickets.index.unique())
colors = colors.dropna()

# find red values
red_values = colors[(colors.str.contains('RED'))].values

# find black values
black_values = colors[colors.str.contains('BLK') | (colors == 'BLACK')].values

# create color mask
color_mask = np.concatenate([red_values, black_values])

# locate most common makes with red/blue-like index
tickets.loc[tickets.index.isin(color_mask), 'Vehicle Make'].value_counts().head()

Vehicle Make
HONDA    73072
TOYOT    65645
NISSA    65048
FORD     53048
CHEVR    39434
Name: count, dtype: int64

### Exercise 21b

In [215]:
# change index back to Issue Date
tickets = tickets.reset_index()
tickets = tickets.set_index('Issue Date')

In [216]:
# 3 car makes most ticketed from Jan 2-10
tickets.sort_index().loc['2020-01-02': '2020-01-10', 'Vehicle Make'].value_counts().head(3)

Vehicle Make
FORD     38936
TOYOT    37073
HONDA    35944
Name: count, dtype: int64

In [217]:
# investigate most ticketed cars in 2020
tickets.loc['2020', 'Plate ID'].value_counts()

# most ticketed plate is not of interest

Plate ID
BLANKPLATE    4288
2704819        572
86107MM        556
NS             498
12125MJ        493
              ... 
HWJ4002          1
HGK4903          1
HTE7388          1
JPB2908          1
M54FDG           1
Name: count, Length: 1896115, dtype: int64

In [218]:
# obtain license plate number most ticketed
most_ticketed = tickets.loc['2020', 'Plate ID'].value_counts().index[1]

# reset index
tickets = tickets.reset_index()

In [219]:
# ensure datetime datatype
tickets['Issue Date'] = pd.to_datetime(tickets['Issue Date'])

# locate entries with most ticketed license plate in 2020
most_ticketed_df = tickets.loc[(tickets['Issue Date'].dt.year==2020) & (tickets['Plate ID']==most_ticketed), :]
most_ticketed_df

,Issue Date,Vehicle Color,Plate ID,Registration State,Vehicle Make,Street Name,Date First Observed
7427786,2020-01-02,WHITE,2704819,IN,FRUEH,9th Ave,0
7427787,2020-01-02,WHITE,2704819,IN,FRUEH,9th Ave,0
7428233,2020-01-02,WHITE,2704819,IN,FRUEH,8th Ave,0
7429845,2020-01-02,WHITE,2704819,IN,FRUEH,8th Ave,0
7429846,2020-01-02,WHITE,2704819,IN,FRUEH,8th Ave,0
...,...,...,...,...,...,...,...
10289735,2020-03-12,WHITE,2704819,IN,FRUEH,9TH AVE,0
10333912,2020-03-13,WHITE,2704819,IN,FRUEH,8th Ave,0
10352396,2020-03-13,WHITE,2704819,IN,FRUEH,8TH AVE,0
10413235,2020-03-16,WHITE,2704819,IN,FRUEH,W 34th St,0


In [220]:
# where registered?
print(most_ticketed_df['Registration State'].value_counts(), '\n')

# always ticketed same spot?
print(most_ticketed_df['Street Name'].value_counts().head(5))

Registration State
IN    571
NJ      1
Name: count, dtype: int64 

Street Name
8th Ave      128
Penn Plz     104
7th Ave       37
9th Ave       36
W 39th St     24
Name: count, dtype: int64


## Exercise 22 - State SAT scores

In [222]:
# load in CSV, specify columns
sat = pd.read_csv(r'D:\Documents\Data Analysis\Datasets\pandas-workout-data\data\sat-scores.csv',
                     usecols=['Year', 'State.Code', 'Total.Math', 'Total.Test-takers', 'Total.Verbal'])
sat.head()

,Year,State.Code,Total.Math,Total.Test-takers,Total.Verbal
0,2005,AL,559,3985,567
1,2005,AK,519,3996,523
2,2005,AZ,530,18184,526
3,2005,AR,552,1600,563
4,2005,CA,522,186552,504


In [223]:
# create multi-index of year and state code
sat = sat.set_index(['Year', 'State.Code'])

In [224]:
# how many took SAT in 2005
sat.loc[2005, 'Total.Test-takers'].sum()

1344824

In [225]:
# average math score in 2010 from NY, NJ, MA, IL
sat.loc[(2010, ['NY', 'NJ', 'MA', 'IL']),'Total.Math'].mean()

535.25

In [226]:
# average verbal score 2012-2015 from AZ, CA, TX
sat.loc[(range(2012, 2016), ['AZ', 'CA', 'TX']), 'Total.Verbal'].mean()

497.3333333333333

### Exercise 22b

In [228]:
# avg math and verbal scores FL, IN, ID across all years
sat.loc[(slice(None), ['FL', 'IN', 'ID']), ['Total.Math', 'Total.Verbal']].mean()

Total.Math     507.09
Total.Verbal   504.61
dtype: float64

In [229]:
# which state received highest verbal score, in which year?
sat.loc[sat['Total.Verbal'].idxmax(), :]

Total.Math           613
Total.Test-takers    174
Total.Verbal         612
Name: (2013, ND), dtype: int64

In [230]:
# avg math score 2005 higher or lower than in 2015?
if sat.loc[2005, 'Total.Math'].mean() > sat.loc[2015, 'Total.Math'].mean():
    print('2005 was higher')
else:
    print('2015 was higher')

2005 was higher


## Exercise 23 - Olympic games

In [232]:
# load in CSV, specify columns, create multi-index
olympics = pd.read_csv(r'D:\Documents\Data Analysis\Datasets\pandas-workout-data\data\olympic_athlete_events.csv',
                     usecols='Age Height Team Year Season City Sport Event Medal'.split(),
                     index_col='Year Season Sport Event'.split())
olympics.head()

,,,,Age,Height,Team,City,Medal
Year,Season,Sport,Event,,,,,
1992,Summer,Basketball,Basketball Men's Basketball,24.00,180.00,China,Barcelona,NaN
2012,Summer,Judo,Judo Men's Extra-Lightweight,23.00,170.00,China,London,NaN
1920,Summer,Football,Football Men's Football,24.00,NaN,Denmark,Antwerpen,NaN
1900,Summer,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,34.00,NaN,Denmark/Sweden,Paris,Gold
1988,Winter,Speed Skating,Speed Skating Women's 500 metres,21.00,185.00,Netherlands,Calgary,NaN


In [233]:
# sort index
olympics = olympics.sort_index()

In [234]:
# avg age of winning athletes in summer games between 1936 and 2000
olympics.loc[(~olympics['Medal'].isna()), :].loc[(slice(1936,2001), 'Summer'), 'Age'].mean()

25.224765104194006

In [235]:
# which team won most medals in all archery events?
olympics.loc[(~olympics['Medal'].isna()), :].loc[(slice(None), slice(None), 'Archery'),'Team'].value_counts().head()

Team
South Korea      69
Belgium          52
France           48
United States    41
China            19
Name: count, dtype: int64

In [236]:
# from 1980 on, avg height of 'Table Tennis Women's Team' event
olympics.loc[(slice(1980,np.inf), slice(None), slice(None), "Table Tennis Women's Team"), 'Height'].mean()

165.04827586206898

In [237]:
# from 1980 on, avg height of above and "Table Tennis Men's Team"?
olympics.loc[(slice(1980,np.inf), slice(None), slice(None), ["Table Tennis Women's Team", "Table Tennis Men's Team"]), 'Height'].mean()

171.26643598615917

In [238]:
# tallest tennis player from 1980 until 2016
olympics.loc[(slice(1980, 2016), slice(None), 'Tennis'), 'Height'].max()

208.0

### Exercise 23b

In [240]:
olympics.head()

Age  Height  \
Year Season Sport     Event                                        
1896 Summer Athletics Athletics Men's 1,500 metres 24.00     NaN   
                      Athletics Men's 1,500 metres   NaN     NaN   
                      Athletics Men's 1,500 metres 22.00     NaN   
                      Athletics Men's 1,500 metres 23.00  154.00   
                      Athletics Men's 1,500 metres 21.00     NaN   

                                                             Team    City  \
Year Season Sport     Event                                                 
1896 Summer Athletics Athletics Men's 1,500 metres  United States  Athina   
                      Athletics Men's 1,500 metres         Greece  Athina   
                      Athletics Men's 1,500 metres      Australia  Athina   
                      Athletics Men's 1,500 metres        Germany  Athina   
                      Athletics Men's 1,500 metres         Greece  Athina   

                                                     Medal  
Year Season Sport     Event                                 
1896 Summer Athletics Athletics Men's 1,500 metres  Silver  
                      Athletics Men's 1,500 metres     NaN  
                      Athletics Men's 1,500 metres    Gold  
                      Athletics Men's 1,500 metres     NaN  
                      Athletics Men's 1,500 metres     NaN

In [241]:
# city that gave out most gold medals from 1980 onward
olympics.loc[olympics['Medal']=='Gold'].loc[1980:, 'City'].value_counts().head()

City
Beijing           671
Rio de Janeiro    665
Athina            664
Sydney            663
London            632
Name: count, dtype: int64

In [242]:
# how many gold medals received by US since 1980
olympics.loc[(olympics['Medal']=='Gold') & (olympics['Team']=='United States')].loc[1980:, 'Medal'].value_counts()

Medal
Gold    1257
Name: count, dtype: int64

## Exercise 24 - Olympic pivots

In [244]:
# load in CSV, specify columns
olympics = pd.read_csv(r'D:\Documents\Data Analysis\Datasets\pandas-workout-data\data\olympic_athlete_events.csv',
                     usecols='Age Height Team Year Season Sport Medal'.split())
olympics.head()

,Age,Height,Team,Year,Season,Sport,Medal
0,24.00,180.00,China,1992,Summer,Basketball,NaN
1,23.00,170.00,China,2012,Summer,Judo,NaN
2,24.00,NaN,Denmark,1920,Summer,Football,NaN
3,34.00,NaN,Denmark/Sweden,1900,Summer,Tug-Of-War,Gold
4,21.00,185.00,Netherlands,1988,Winter,Speed Skating,NaN


In [245]:
# only include games 1980 to present
olympics = olympics.loc[olympics['Year'] >= 1980, :]

# only include specific countries
olympics = olympics.loc[olympics['Team'].isin(['Great Britain', 'France', 'United States', 'Switzerland', 'China', 'India']),:]

In [246]:
olympics.head()

,Age,Height,Team,Year,Season,Sport,Medal
0,24.00,180.00,China,1992,Summer,Basketball,NaN
1,23.00,170.00,China,2012,Summer,Judo,NaN
10,31.00,188.00,United States,1992,Winter,Cross Country Skiing,NaN
11,31.00,188.00,United States,1992,Winter,Cross Country Skiing,NaN
12,31.00,188.00,United States,1992,Winter,Cross Country Skiing,NaN


In [247]:
# average age
olympics['Age'].mean()

25.201190860504322

In [248]:
# which countries have consistently youngest athletes
olympics.pivot_table(index='Year',
                     columns='Team',
                     values='Age',
                     aggfunc='min')

Team,China,France,Great Britain,India,Switzerland,United States
Year,,,,,,
1980,14.00,16.00,13.00,15.00,15.00,15.00
1984,15.00,14.00,13.00,16.00,15.00,15.00
1988,11.00,16.00,15.00,16.00,19.00,14.00
1992,13.00,13.00,15.00,18.00,17.00,14.00
1994,13.00,16.00,18.00,NaN,16.00,16.00
1996,13.00,14.00,16.00,16.00,15.00,14.00
1998,16.00,18.00,18.00,16.00,16.00,15.00
2000,14.00,16.00,16.00,17.00,15.00,15.00
2002,17.00,17.00,19.00,20.00,16.00,16.00


In [249]:
# how tall were tallest athletes in each sport in each year
olympics.pivot_table(index='Sport',
                     columns='Year',
                     values='Height',
                     aggfunc='max',
                     sort=True)

Year,1980,1984,1988,1992,1994,1996,1998,2000,2002,2004,2006,2008,2010,2012,2014,2016
Sport,,,,,,,,,,,,,,,,
Alpine Skiing,184.00,184.00,185.00,185.00,188.00,NaN,188.00,NaN,189.00,NaN,193.00,NaN,193.00,NaN,200.00,NaN
Archery,185.00,188.00,188.00,191.00,NaN,191.00,NaN,191.00,NaN,193.00,NaN,193.00,NaN,193.00,NaN,188.00
Athletics,197.00,203.00,203.00,200.00,NaN,198.00,NaN,197.00,NaN,203.00,NaN,203.00,NaN,208.00,NaN,203.00
Badminton,NaN,NaN,NaN,186.00,NaN,189.00,NaN,187.00,NaN,190.00,NaN,190.00,NaN,191.00,NaN,191.00
Baseball,NaN,NaN,NaN,198.00,NaN,195.00,NaN,206.00,NaN,NaN,NaN,198.00,NaN,NaN,NaN,NaN
Basketball,196.00,216.00,216.00,216.00,NaN,216.00,NaN,226.00,NaN,226.00,NaN,226.00,NaN,221.00,NaN,218.00
Beach Volleyball,NaN,NaN,NaN,NaN,NaN,193.00,NaN,195.00,NaN,192.00,NaN,202.00,NaN,202.00,NaN,188.00
Biathlon,190.00,190.00,188.00,192.00,192.00,NaN,192.00,NaN,192.00,NaN,193.00,NaN,193.00,NaN,193.00,NaN
Bobsleigh,NaN,184.00,NaN,NaN,NaN,NaN,198.00,NaN,190.00,NaN,193.00,NaN,191.00,NaN,189.00,NaN


In [250]:
# how many medals each country earn each year
olympics.pivot_table(index='Year',
                     columns='Team',
                     values='Medal',
                     aggfunc='count')

Team,China,France,Great Britain,India,Switzerland,United States
Year,,,,,,
1980,0.00,30.00,48.00,16.00,5.00,30.00
1984,74.00,70.00,71.00,0.00,17.00,359.00
1988,50.00,31.00,54.00,0.00,24.00,212.00
1992,73.00,65.00,50.00,0.00,2.00,236.00
1994,3.00,11.00,3.00,NaN,8.00,19.00
1996,94.00,49.00,26.00,1.00,11.00,255.00
1998,14.00,13.00,4.00,0.00,10.00,30.00
2000,65.00,64.00,52.00,1.00,14.00,240.00
2002,14.00,13.00,6.00,0.00,20.00,70.00


### Exercise 24b

In [252]:
# create pivot table for number of medals each team won per year, index displaying year and season
olympics.pivot_table(index=['Year', 'Season'],
                     columns='Team',
                     values='Medal',
                     aggfunc='count')

Team         China  France  Great Britain  India  Switzerland  United States
Year Season                                                                 
1980 Summer    NaN   29.00          47.00  16.00         2.00            NaN
     Winter   0.00    1.00           1.00    NaN         3.00          30.00
1984 Summer  74.00   67.00          71.00   0.00        13.00         352.00
     Winter   0.00    3.00           0.00    NaN         4.00           7.00
1988 Summer  50.00   29.00          54.00   0.00         8.00         207.00
     Winter   0.00    2.00           0.00   0.00        16.00           5.00
1992 Summer  70.00   55.00          50.00   0.00         1.00         222.00
     Winter   3.00   10.00           0.00   0.00         1.00          14.00
1994 Winter   3.00   11.00           3.00    NaN         8.00          19.00
1996 Summer  94.00   49.00          26.00   1.00        11.00         255.00
1998 Winter  14.00   13.00           4.00   0.00        10.00          30.00
2000 Summer  65.00   64.00          52.00   1.00        14.00         240.00
2002 Winter  14.00   13.00           6.00   0.00        20.00          70.00
2004 Summer  82.00   53.00          55.00   1.00         5.00         259.00
2006 Winter   9.00   15.00           1.00   0.00        15.00          48.00
2008 Summer 170.00   77.00          81.00   3.00        11.00         309.00
2010 Winter  15.00   14.00           1.00   0.00        12.00          89.00
2012 Summer 117.00   78.00         122.00   6.00         4.00         238.00
2014 Winter  12.00   18.00          10.00   0.00        28.00          52.00
2016 Summer 109.00   96.00         145.00   2.00        11.00         256.00

In [253]:
# pivot table with average age and height per year, per team
olympics.pivot_table(index='Year',
                     columns='Team',
                     values=['Age', 'Height'])

Age                                                      Height         \
Team China France Great Britain India Switzerland United States  China France   
Year                                                                            
1980 21.87  23.52         22.88 25.51       24.56         22.77 162.64 175.10   
1984 22.08  24.37         24.45 24.91       23.59         24.44 171.61 176.28   
1988 22.36  24.52         25.44 24.00       26.22         24.90 171.30 176.16   
1992 21.96  25.14         25.58 24.18       25.41         25.47 171.00 175.23   
1994 20.63  24.60         25.28   NaN       25.50         24.98 169.23 173.01   
1996 22.02  25.30         26.75 24.63       27.12         26.27 170.40 174.49   
1998 21.78  25.46         27.24 16.00       25.64         25.15 168.91 174.83   
2000 22.52  25.98         26.41 25.40       27.38         26.58 170.96 176.23   
2002 23.13  25.74         26.83 20.00       26.24         25.73 169.01 175.77   
2004 23.01  26.14         26.30 24.73       27.34         26.44 172.75 176.11   
2006 23.46  26.30         26.85 25.20       26.28         25.64 170.31 174.04   
2008 23.90  26.29         25.20 25.40       27.31         26.23 176.02 176.96   
2010 23.24  25.91         26.15 25.67       26.55         25.84 169.56 174.77   
2012 23.89  26.61         25.92 25.64       27.17         26.46 175.45 177.34   
2014 23.40  25.71         25.63 25.00       25.86         26.19 169.81 173.77   
2016 23.87  27.10         26.65 26.10       25.89         26.22 174.99 177.75   

                                                     
Team Great Britain  India Switzerland United States  
Year                                                 
1980        175.44 170.75      177.01        175.02  
1984        174.41 169.40      173.42        176.88  
1988        176.08 168.70      176.61        176.62  
1992        176.15 172.50      175.33        176.40  
1994        172.92    NaN      174.11        175.44  
1996        177.44 171.79      174.11        177.48  
1998        173.76 183.00      174.28        173.47  
2000        176.48 173.33      176.49        177.33  
2002        173.06 183.00      176.19        173.77  
2004        176.66 169.91      176.31        176.94  
2006        176.11 167.80      176.01        173.87  
2008        176.89 173.12      176.05        177.47  
2010        176.34 176.00      176.62        174.49  
2012        177.76 173.57      177.10        178.09  
2014        173.43 170.00      174.05        174.71  
2016        176.40 169.97      173.86        177.79

In [254]:
# pivot table average age and height per year, per team, broken up by both year and season
olympics.pivot_table(index=['Year', 'Season'],
                     columns='Team',
                     values=['Age', 'Height'])

Age                                                      Height  \
Team        China France Great Britain India Switzerland United States  China   
Year Season                                                                     
1980 Summer   NaN  23.70         22.76 25.51       25.33           NaN    NaN   
     Winter 21.87  22.75         23.57   NaN       23.48         22.77 162.64   
1984 Summer 22.10  24.30         24.46 24.91       23.65         24.60 172.07   
     Winter 21.99  24.79         24.35   NaN       23.36         23.56 166.29   
1988 Summer 22.46  24.50         25.78 24.07       27.60         25.21 171.28   
     Winter 19.60  24.58         23.42 22.67       24.39         23.62 175.00   
1992 Summer 21.96  25.55         25.86 24.43       26.33         25.63 170.95   
     Winter 21.93  23.95         23.82 20.50       24.07         24.93 171.79   
1994 Winter 20.63  24.60         25.28   NaN       25.50         24.98 169.23   
1996 Summer 22.02  25.30         26.75 24.63       27.12         26.27 170.40   
1998 Winter 21.78  25.46         27.24 16.00       25.64         25.15 168.91   
2000 Summer 22.52  25.98         26.41 25.40       27.38         26.58 170.96   
2002 Winter 23.13  25.74         26.83 20.00       26.24         25.73 169.01   
2004 Summer 23.01  26.14         26.30 24.73       27.34         26.44 172.75   
2006 Winter 23.46  26.30         26.85 25.20       26.28         25.64 170.31   
2008 Summer 23.90  26.29         25.20 25.40       27.31         26.23 176.02   
2010 Winter 23.24  25.91         26.15 25.67       26.55         25.84 169.56   
2012 Summer 23.89  26.61         25.92 25.64       27.17         26.46 175.45   
2014 Winter 23.40  25.71         25.63 25.00       25.86         26.19 169.81   
2016 Summer 23.87  27.10         26.65 26.10       25.89         26.22 174.99   

                                                                   
Team        France Great Britain  India Switzerland United States  
Year Season                                                        
1980 Summer 175.89        175.51 170.75      180.53           NaN  
     Winter 171.42        174.52    NaN      171.59        175.02  
1984 Summer 176.52        174.44 169.40      173.93        177.18  
     Winter 174.64        174.12    NaN      171.38        175.21  
1988 Summer 176.19        176.64 168.70      176.83        177.10  
     Winter 176.06        172.78    NaN      176.30        174.52  
1992 Summer 176.01        176.56 172.50      175.58        176.94  
     Winter 172.96        173.25    NaN      174.97        174.47  
1994 Winter 173.01        172.92    NaN      174.11        175.44  
1996 Summer 174.49        177.44 171.79      174.11        177.48  
1998 Winter 174.83        173.76 183.00      174.28        173.47  
2000 Summer 176.23        176.48 173.33      176.49        177.33  
2002 Winter 175.77        173.06 183.00      176.19        173.77  
2004 Summer 176.11        176.66 169.91      176.31        176.94  
2006 Winter 174.04        176.11 167.80      176.01        173.87  
2008 Summer 176.96        176.89 173.12      176.05        177.47  
2010 Winter 174.77        176.34 176.00      176.62        174.49  
2012 Summer 177.34        177.76 173.57      177.10        178.09  
2014 Winter 173.77        173.43 170.00      174.05        174.71  
2016 Summer 177.75        176.40 169.97      173.86        177.79